## Reddit Crawling

### 크롤링 막힌듯?

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [ ]:
user_agent = user_agent = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

reddit = 'https://www.reddit.com/r/Music/'

tag = []
content = []
date = []

res = requests.get(reddit, user_agent)
soup = BeautifulSoup(res.text, "html.parser")
# driver = webdriver.Chrome()

# driver.get(reddit)
# time.sleep(1)

# page = soup.find("div", "rpBJOHq2PR60pnwJlUyP0")

titles = soup.find_all('h3')
for title in titles:
    print('titile: ', title.text)

time.sleep(1)

contents = soup.find_all(['p', 'li'])
for content in contents:
    print(content.text)

# search = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input')
# ActionChains(driver).send_keys_to_element(search, "hip pop").perform(1)
# search.send_keys(Keys.RETURN)


Your request has been blocked due to a network policy.
Try logging in or creating an account here to get back to browsing.
If you're running a script or application, please register or sign in with your developer credentials here. Additionally make sure your User-Agent is not empty and is something unique and descriptive and try again. if you're supplying an alternate User-Agent string,
try changing back to default as that can sometimes result in a block.
You can read Reddit's Terms of Service here.
if you think that we've incorrectly blocked you or you would like to discuss
easier ways to get the data you want, please file a ticket here.
when contacting us, please include your ip address which is: 34.42.94.142 and reddit account


### reddit API 사용

In [ ]:
import praw

reddit = praw.Reddit(
    client_id='***',
    client_secret='***',
    user_agent='bokyungg',
    username='bokyungg',
    password='***'
)

In [ ]:
next_key = 1
red = {}

def get_content_reddit(subred, keyword):
    subreddit = reddit.subreddit(subred)
    for submission in subreddit.hot(limit=1000):
        title = submission.title
        content = submission.selftext
        if len(content) > 0:
            if keyword in content.lower():
                add_item(red, {'title': title, 'content': content})
        if len(red) == 100:
            break

In [ ]:
get_content_reddit('Music', 'r&b')
red

{1: {'title': 'any alternative music about Jesus/God? (do not include flyleaf or any christian metal)',
  'content': 'any alternative music about Jesus/God? i know about flyleaf, stryper, extol, etc. but im looking for a more indie rock/gothic/shoegaze/dream pop/alt contemporary r&b sounding…literally all those genres i listed! if u have any in mind, any bands, anyone (do not include gospel) thank u.\n\nedit: ohhhh, some song examples i wanna hear that’s in the genre im talking about are; song to a siren by this mortal coil. alison by slowdive. shes so high by tal bachman. from the start by laufey. lullaby by the cure. etc…it’s probably hard to find but ik its out there, thank u ♡ '},
 2: {'title': 'What to know/expect at an R&B concert?',
  'content': "I've never gone to a concert before, but Mahalia is coming to NYC in March and I'm in close enough proximity to go without travel being much of a hassle. I'm on the fence about it because I'd probably have to go on my own, it'd be my fi

In [ ]:
red_df = pd.DataFrame(red)
red_df = red_df.transpose()

In [ ]:
red_df.head()

,title,content
101,In need of some new music,I hope this kind of post is okay here. I feel ...
102,"Music that has a ""tropical"" tone?",Curious if anyone can recommend any songs that...
103,Trying to listen to 365 Albums in 2024,I listen to music pretty religiously and reall...
104,I (nonsensically) calculated who the loser is ...,Video: [Loser](https://www.youtube.com/watch?v...
105,Listened to MJs off the wall for the first time,I was a baby in 82. I never heard the entiret...


### 글 내용 감정분석

- 간단하게 textblob으로 0~5 사이로 나타냄

In [ ]:
from textblob import TextBlob

def analyze_sentiment(red):
    analysis = TextBlob(red)

    # 0-5 범위로 변환
    sentiment_score = (analysis.sentiment.polarity + 1) * 2.5
    return sentiment_score

In [ ]:
red_df['sentiment_score'] = red_df['content'].apply(analyze_sentiment)
red_df.head()

,title,content,sentiment_score
101,In need of some new music,I hope this kind of post is okay here. I feel ...,3.462730
102,"Music that has a ""tropical"" tone?",Curious if anyone can recommend any songs that...,2.629464
103,Trying to listen to 365 Albums in 2024,I listen to music pretty religiously and reall...,2.930556
104,I (nonsensically) calculated who the loser is ...,Video: [Loser](https://www.youtube.com/watch?v...,2.491587
105,Listened to MJs off the wall for the first time,I was a baby in 82. I never heard the entiret...,2.646528


In [ ]:
red_df.to_parquet('reddit.parquet', index=False)